<a href="https://colab.research.google.com/github/fltenwall/MasterStudy/blob/main/Copy_of_select_number_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np

def bibd_sample(items, k):
    sample = np.copy(items)
    first_select = []
    second_select = []
    third_select = []
    # 100維
    label = 0
    first_dim = 100
    second_dim = 50
    third_dim = 50
    while label < 99:

      random_number1 = np.random.randint(0,first_dim-1)

      # 第一個隨機數已經產生過，則繼續生成新的隨機數
      while random_number1 in first_select and len(first_select) <= first_dim:
        random_number1 = np.random.randint(0,first_dim-1)

      if len(first_select) <= first_dim:
        first_select.append(random_number1)

      index2 = 0
      while index2 < 5:
        random_number2 = np.random.randint(0,second_dim-1)
        # 對每一行取出5個樣本，每個樣本本身是50個格子
        while random_number2 in second_select and len(second_select) <= 5:
          random_number2 = np.random.randint(0,second_dim-1)
        if len(second_select) <= 5:
          second_select.append(random_number2)

          index3 = 0
          # 在每一個格子中取出10個樣本
          while index3 < 10:
            random_number3 = np.random.randint(0,third_dim-1)
            while random_number3 in third_select and len(third_select) <= 10:
              random_number3 = np.random.randint(0,third_dim-1)
            sample[random_number1][random_number2][random_number3][3] = 1
            index3 = index3 + 1
          third_select = [] # 重置

        index2 = index2 + 1
      second_select = []

      label = label + 1
    return sample

items = []
for i in range(100):
  arr1 = []
  for j in range(50):
    arr2 = []
    for k in range(50):
      arr2.append([i, j, k, 0])
    # 隨機打亂術組
    #random.shuffle(arr2)
    arr1.append(arr2)
  # 隨機打亂術組
  #random.shuffle(arr1)
  items.append(arr1)
# 隨機打亂術組
#items = random.shuffle(items)

k = 10000  # 我们需要抽取10000个样本
sample = bibd_sample(items, k)



In [ ]:
# expend
result = [k for i in sample for j in i for k in j]
random.shuffle(result)

In [ ]:
result[0:9] # [i, j, k, value]

In [ ]:
# 從 i 統計出現1對格子
x1 = 0
for i in range(99):
  for item in result:
    if(item[0] == i):
      x1 = x1 + item[3]
print(x1)

In [ ]:
# 從 j 統計出現1對格子
x2 = 0
for i in range(50):
  for item in result:
    if(item[1] == i):
      x2 = x2 + item[3]
print(x2)

In [ ]:
# 從 k 統計出現1對格子
x3 = 0
for i in range(50):
  for item in result:
    if(item[2] == i):
      x3 = x3 + item[3]
print(x3)

In [ ]:
# 對全體樣本統計出現1的格子
x4 = 0
for item in result:
    x4 = x4 + item[3]
print(x4)

In [ ]:
items = []
for i in range(100):
  arr1 = []
  for j in range(100):
    arr2 = []
    for k in range(100):
      arr2.append([i, j, k, 0])
    arr1.append(arr2)
  items.append(arr1)
sample = np.copy(items)
sample

In [ ]:
import numpy as np

In [ ]:
# matrix = np.random.rand(100,50,50)

In [ ]:
arr = np.zeros((100,50,50))

In [ ]:
np.bincount(best_ind[:5000], minlength=100)

In [1]:
# 遗传算法
import numpy as np
from deap import base, creator, tools, algorithms
import random

# 初始化三维矩阵
X = np.zeros((100, 50, 50))

# 定义选择的数量
N = 5000

# 定义适应度函数
def fitness_function(ind):
    # 提取选择的索引
    i_ind = ind[:N]
    j_ind = ind[N:2*N]
    k_ind = ind[2*N:3*N]
    
    # 计算频数
    freq_i = np.bincount(i_ind, minlength=100)
    freq_j = np.bincount(j_ind, minlength=50)
    freq_k = np.bincount(k_ind, minlength=50)

    # 合并频数
    combined_freq = np.concatenate((freq_i, freq_j, freq_k))
    
    # 返回最小频数（作为适应度，取负值进行最大化）
    return -np.min(combined_freq),

# 遗传算法参数设置
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # 最小化
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.choices, range(100), k=N)  # i 的选择
toolbox.register("j_indices", random.choices, range(50), k=N)  # j 的选择
toolbox.register("k_indices", random.choices, range(50), k=N)  # k 的选择
toolbox.register("individual", tools.initIterate, creator.Individual, 
                 lambda: toolbox.indices() + toolbox.j_indices() + toolbox.k_indices())
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# 遗传算法执行
population = toolbox.population(n=500)  # 种群大小
ngen = 100  # 最大迭代次数

for gen in range(ngen):
    # 选择
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # 交叉和变异
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:  # 交叉概率
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.3:  # 变异概率
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 评估新一代个体
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # 替换种群
    population[:] = offspring

# 输出最佳解
fits = [ind.fitness.values[0] for ind in population]
best_idx = np.argmax(fits)
best_ind = population[best_idx]

# 计算最佳解的最小频数
best_freq_i = np.bincount(best_ind[:N], minlength=100)
best_freq_j = np.bincount(best_ind[N:2*N], minlength=50)
best_freq_k = np.bincount(best_ind[2*N:3*N], minlength=50)
combined_best_freq = np.concatenate((best_freq_i, best_freq_j, best_freq_k))
min_freq = np.min(combined_best_freq)

print("最佳最小频数:", min_freq)

最佳最小频数: 27


In [1]:
import numpy as np
import random
from itertools import combinations

def generate_bibd(v, k, r, lambda_):
    """生成平衡不重复设计（BIBD）的函数"""
    # v: 总元素数
    # k: 每个块的大小
    # r: 每个元素的重复次数
    # lambda_: 每对元素出现的次数
    b = (v * r) // k  # 块的数量
    # 生成所有元素的组合
    elements = list(range(v))
    blocks = []
    
    for combo in combinations(elements, k):
        blocks.append(combo)
    
    # 过滤出符合 BIBD 条件的块
    selected_blocks = []
    for block in blocks:
        counts = [0] * v
        for b in selected_blocks:
            for elem in b:
                counts[elem] += 1
        if all(counts[elem] < r for elem in block):
            selected_blocks.append(block)
            if len(selected_blocks) == b:
                break
    
    return selected_blocks

def select_elements_from_bibd(v, k, r, lambda_, total_selection):
    """从 BIBD 中选择元素"""
    blocks = generate_bibd(v, k, r, lambda_)
    selected_indices = []
    
    for block in blocks:
        selected_indices.extend(random.sample(block, min(k, total_selection - len(selected_indices))))
        if len(selected_indices) >= total_selection:
            break
    
    return selected_indices[:total_selection]

# 初始化三维矩阵
X = np.zeros((100, 50, 50))

# 定义选择的参数
N = 5000  # 选择的总数
v = 100   # 总元素数
k = 5     # 每个块的大小
r = 50    # 每个元素的重复次数
lambda_ = 1  # 每对元素出现的次数

# 使用 BIBD 方法选择元素
selected_indices = select_elements_from_bibd(v, k, r, lambda_, N)

# 计算频数
freq = np.bincount(selected_indices, minlength=v)

# 输出频数
print("频数表:", freq)

# 计算最小频数
min_freq = np.min(freq)
print("最小频数:", min_freq)


频数表: [50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 42 36
 32 30 27 23]
最小频数: 23


In [3]:
import pandas as pd
import numpy as np
import random

# 初始化三维矩阵
X = np.zeros((100, 50, 50))

In [26]:
for i in range(100):
    for j in range(50):
        for k in range(50):
            if i == j or j == k or i == k:
                X[i,j,k] = 1

In [30]:
rows = np.sum(X,axis=1)
rows.min()

1.0